In [20]:
import scipy.io
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu

1.	Implemente una función que reciba una señal de múltiples canales y épocas y calcule la Energía de promedio de cada canal.

In [21]:
# Ruta de la carpeta
carpeta_control = "./datos_senales_datos_parkinson_cursos/control"
carpeta_parkinson = "./datos_senales_datos_parkinson_cursos/parkinson"

# Obtener los nombres de los archivos en la carpeta
control = os.listdir(carpeta_control)
parkinson = os.listdir(carpeta_parkinson)
directorios = {'control': [carpeta_control + f'/{dir}' for dir in control], 
                    'parkinson': [carpeta_parkinson + f'/{dir}' for dir in parkinson]}


In [22]:
def prom_canal(signal):
    """
    Calcula el promedio de los datos.
    
    Parámetros:
    signal (numpy.ndarray): La señal completa.
    
    Retorna:
    promedios: El promedio de los datos para cada.
    """

    canales = signal.shape[0]
    epocas = signal.shape[1]

    promedios = {}
    for canal in range(canales):
        energias = []
        for epoca in range(epocas):
            energia = np.sum(signal[canal][epoca]**2)
            energias.append(energia)
        
        promedio = np.mean(energias)
        promedios[canal + 1] = promedio

    return promedios

In [23]:
signal_test = scipy.io.loadmat(directorios['control'][0])['data']
canales, datos, epocas = signal_test.shape

print(f"""\tForma de la señal.
Canales: {canales}
Datos: {datos}
Epocas: {epocas}
""")

for promedio in prom_canal(signal_test).items():
    print(f'Canal: {promedio[0]} Promedio: {promedio[1]}')

	Forma de la señal.
Canales: 8
Datos: 2000
Epocas: 180

Canal: 1 Promedio: 1931.9085322346082
Canal: 2 Promedio: 1888.731712081874
Canal: 3 Promedio: 2048.413462942386
Canal: 4 Promedio: 1665.5076255269491
Canal: 5 Promedio: 2675.714672322914
Canal: 6 Promedio: 2271.9742265301916
Canal: 7 Promedio: 2050.3194828578053
Canal: 8 Promedio: 2219.2739560497716


2.	Calcule la energía de cada canal promediada por épocas para cada sujeto, esto para ambos grupos poblacionales. Guarde esta información en un DataFrame de columnas ‘canal’ y filas ‘#sujeto’ con los valores de energía calculados, un DataFrame para cada grupo poblacional. 

In [24]:
dic_control = {}
dic_parkinson = {}

for control in directorios['control']:
    dic_control[control[48:51]] = prom_canal(scipy.io.loadmat(control)['data'])

for parkinson in directorios['parkinson']:
    dic_parkinson[parkinson[50:53]] = prom_canal(scipy.io.loadmat(parkinson)['data'])

df_control = pd.DataFrame(dic_control).T
df_parkinson = pd.DataFrame(dic_parkinson).T


3.	Determine si existe diferencia estadística entre canales de cada grupo de sujetos a través de una prueba t. Compruebe los supuestos necesarios para realizar una prueba t, esto es: Normalidad de la variable, independencia (se asume que los grupos son independientes), y homocedasticidad (use una prueba de Levene), finalmente realice la prueba t para determinar si existen diferencias entre los canales entre grupos de sujetos. De no cumplirse los requisitos, realice entonces un análisis no paramétrico (prueba U de Mann-Whitney). Este numeral tiene como objetivo identificar los canales que entregan información diferencial entre pacientes Sanos y con enfermedad de Parkinson.

In [25]:
def estacionariedad(canal1, canal2):
  print("\nPrueba de normalidad de Shapiro-Wilk")
  shapiro1 = shapiro(canal1)
  shapiro2 = shapiro(canal2)

  print(f"Canal 1: p = {shapiro1.pvalue}")
  print(f"Canal 2: p = {shapiro2.pvalue}")

  print("Prueba de homocedasticidad de Levene")
  levene_test = levene(canal1, canal2)
  print(f"p = {levene_test.pvalue}")

  normal1 = shapiro1.pvalue > 0.05
  normal2 = shapiro2.pvalue > 0.05
  homocedasticos = levene_test.pvalue > 0.05

  print("Prueba de comparación entre ciclos")
  if normal1 and normal2 and homocedasticos:
      print("Ambos canales son normales y homocedásticos")
      t_test = ttest_ind(canal1, canal2)
      print(f"t-test: p = {t_test.pvalue}")
      diferencia = mw_test.pvalue < 0.05
      print(f"Diferencia: {diferencia}")

  else:
      print("Se incumplen supuestos:")
      print(f"Normalidad: Ciclo 1: {normal1}, Ciclo 2: {normal2}")
      print(f"Homocedasticidad: {homocedasticos}")
      mw_test = mannwhitneyu(canal1, canal2)
      print(f"Mann-Whitney U: p = {mw_test.pvalue}")
      diferencia = mw_test.pvalue < 0.05
      print(f"Diferencia: {diferencia}")

In [26]:
n_canales = 8

for i in range(n_canales):
    print('\n\tCanal', i + 1)
    estacionariedad(df_control[i + 1].values, df_parkinson[i + 1].values)


	Canal 1

Prueba de normalidad de Shapiro-Wilk
Canal 1: p = 0.018734201543374754
Canal 2: p = 0.010232982026788719
Prueba de homocedasticidad de Levene
p = 0.9131972737769418
Prueba de comparación entre ciclos
Se incumplen supuestos:
Normalidad: Ciclo 1: False, Ciclo 2: False
Homocedasticidad: True
Mann-Whitney U: p = 0.8581496425719736
Diferencia: False

	Canal 2

Prueba de normalidad de Shapiro-Wilk
Canal 1: p = 0.01746312958074201
Canal 2: p = 0.004192413269198882
Prueba de homocedasticidad de Levene
p = 0.6416812420608209
Prueba de comparación entre ciclos
Se incumplen supuestos:
Normalidad: Ciclo 1: False, Ciclo 2: False
Homocedasticidad: True
Mann-Whitney U: p = 0.993799802489995
Diferencia: False

	Canal 3

Prueba de normalidad de Shapiro-Wilk
Canal 1: p = 0.014804755276209738
Canal 2: p = 0.0031016484454348407
Prueba de homocedasticidad de Levene
p = 0.640381496752782
Prueba de comparación entre ciclos
Se incumplen supuestos:
Normalidad: Ciclo 1: False, Ciclo 2: False
Homoceda